# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import datetime
from scipy.stats import linregress
import api_keys
from pprint import pprint


# Import API key(from weather and google api)
from api_keys import g_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
# Load the csv from output data file (Part I)
output_file = "../cities_output2.csv"


# Create DataFrame for output data file from cities/weather (Part I)
output_df = pd.read_csv(r'../cities_output2.csv')
output_df.head() 


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp (F),Wind Speed
0,hilo,90,US,1627242314,85,19.7297,-155.0900,81.81,4.61
1,jamestown,40,US,1627242056,57,42.0970,-79.2353,85.32,9.73
2,vaini,20,TO,1627242314,94,-21.2000,-175.2000,68.16,0.00
3,port alfred,58,ZA,1627242314,43,-33.5906,26.8910,61.18,11.21
4,dikson,100,RU,1627242310,85,73.5069,80.5464,39.42,14.88


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Define locations
locations = output_df[["Lat", "Lng"]]

# Define humidity
#humidity = output_df["Humidity"].astype(float)
humidity = output_df["Humidity"]

#fig = gmaps.figure()
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=3)
max_intensity = np.max(humidity)

# Create heat layer

#heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=5, point_radius=1)
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=4)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
#Define first Dataframe, focusing on Max Temp, Wind Speed, and Cloudines

# Create base DataFrame
output_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp (F),Wind Speed
0,hilo,90,US,1627242314,85,19.7297,-155.0900,81.81,4.61
1,jamestown,40,US,1627242056,57,42.0970,-79.2353,85.32,9.73
2,vaini,20,TO,1627242314,94,-21.2000,-175.2000,68.16,0.00
3,port alfred,58,ZA,1627242314,43,-33.5906,26.8910,61.18,11.21
4,dikson,100,RU,1627242310,85,73.5069,80.5464,39.42,14.88


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
# Isolate cities with ideal conditions - wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
ideal_city_df = output_df.loc[(output_df["Wind Speed"] <= 10) & (output_df["Cloudiness"] == 0) & \
                                   (output_df["Max Temp (F)"] >= 70) & (output_df["Max Temp (F)"] <= 80)].dropna()

ideal_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp (F),Wind Speed
128,kinango,0,KE,1627242345,82,-4.1372,39.3153,73.51,6.49
197,yima,0,CN,1627242364,85,34.7381,111.8839,73.44,1.70
288,lithakia,0,GR,1627242389,50,37.7167,20.8333,79.88,1.14
304,jishou,0,CN,1627242393,90,28.3167,109.7167,73.83,1.43
349,itambacuri,0,BR,1627242405,55,-18.0311,-41.6850,72.23,9.04


In [32]:
#Import API key
from api_keys import g_key

# Configure gmaps; addressing issues with g_key
gmaps.configure(api_key = g_key)

# Create hotel file
hotel_df = ideal_city_df.reset_index(drop=True)

# Add Hotel Name column
hotel_df["Hotel Name"] = ""

#Define params
params = {
    "radius": 5000,
    "types" : "hotel",
    "keyword" : "hotels",
    "key" : g_key
}
    
# Loop and use lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # Define latitudes and longitudes
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Maintain parameters with various locations
    params["location"] = f"{lat},{lng}"
    
    # Define url
    main_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Capture hotel name
    hotel_name = requests.get(main_url, params = params)
    
    # Convert to json file
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Include option to skip missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing result.... Skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing result.... Skipping.
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 34.7473615,
                    "lng": 111.8732663
                },
                "viewport": {
                    "northeast": {
                        "lat": 34.74866932989271,
                        "lng": 111.8745961798927
                    },
                    "southwest": {
                        "lat": 34.74596967010727,
                        "lng": 111.8718965201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Huanghe Hotel",
            "place_id": "ChIJa2ASoqZSeDYRQd53htxxFhk",
            "plus_code": {
                "compound_code": "PVWF+W

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBLKpXXkJjgXfECGHTx4ucVMfgBk0G136idgnCY8yhBiCSqPOPn9lJIUCEMIYTjuisCDWdBO-5qt5UhZXeXdxSDyPtU6WeMC5ERSVI3K22vWkJz_CbkJjwUR1y65vu_5pBiCu2HlVjK2mkpatxBeh6pHazdKykY1h4KCzJiVf3qeHQDvBGRZ0rZpG3htXnICHw26190G7dc_zz0ISAn_xnk87O6Xw0rDfQfgIcZWNcySRCUYUhBzHQ_iTatU4-GrJnSx44pl91DG-VHO1OeurkO90J0N2perVswgS1GPVHjmESE9PDo4GbXIzL12KNXR0_smKcArcY2YQ-W9Poz2pvTsKj-EsUsmXFM30d12GvcRlthN3lDUdh8HEfk3gdZnUCAvf67AaRUyA6Z2i_VdhOeqap_Cyrd8LobqaUHjxktCcz_evNSehpbM23eq4Oy",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.7218519,
                    "lng": 20.8596087
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.72328272989272,
                        "lng": 20.86101712989272
                    },
                    "southwest": {
                        "lat": 37.720583070

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 28.293232,
                    "lng": 109.730735
                },
                "viewport": {
                    "northeast": {
                        "lat": 28.29463582989272,
                        "lng": 109.7320887798927
                    },
                    "southwest": {
                        "lat": 28.29193617010728,
                        "lng": 109.7293891201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Xiangxi National Hotel",
            "photos": [
                {
                    "height": 1371,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/109112600512780255436\">\u674e\u

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -18.032994,
                    "lng": -41.678824
                },
                "viewport": {
                    "northeast": {
                        "lat": -18.03164167010728,
                        "lng": -41.67747317010728
                    },
                    "southwest": {
                        "lat": -18.03434132989272,
                        "lng": -41.68017282989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Pousada Bela",
            "photos": [
                {
                    "height": 3000,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/114836497580225419797\">sdjsky</a>"
 

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDOB1IB7QrYpiMbBAewu3hS_ktqhgyorpuHuTz06PFdsJW00TCjXyXX5y0JqfB7bzT-THyL5boie0OFi2ZO2NZ2dckAIdzJ5nRH2pZ9aN-hME7PzbtboyZr3H52PGv5y2nTy-Kw8_jSFY1NfefhbheMI34zBGbw-mbDHdwF1SZ4h-JrOa0gDL0tHa8pH1VMhidrnoORu58U4BpTCtOJp5LDWo-71xIMdTvEIMM6ceTicMTNJW_sViHLw5qV7-AruPxAIGM-sNcBs5TCaK11kJdvxyiRlHy076HK0CoqHp7PybSZGkSE44OBrX4Bg9M-gWGABYPL1AyzRMjjG0AvHHPOcxSvTEb6xVk7Bf_m84lwoVh2wTVL6HjS5U0d-UN-fSgk7Ifks3G2ILyIiwJ4kbM_h-Jl23tAWyWPSkaIo2SuAgf2ccF3bS5QhcLpMhim",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -8.8144567,
                    "lng": 13.2350337
                },
                "viewport": {
                    "northeast": {
                        "lat": -8.813115220107276,
                        "lng": 13.23624852989272
                    },
                    "southwest": {
                        "lat": -8.81581487

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAsGHX-WVmraeEoveJaQ2fgvfHmTYCpEuTsuodl7mo0vCmbbECn3pnszL4P9vIKLkobEYkBGo8Imkdr2p1bAkYArC5FcROVx4Wx9voh3npOLYwZQRpwMjGF_S8BAaNHRDZwugComGU89r7Qr388EoApjgfyv8OekZgA2SWZixgnpHDGGu9FnEZul8jHZ1DOQFzhgtm97oEQzKJpqlW27zF5_O6uadiW3dmdJezPq9sxbwbjvGUdMa0iVGQwRrAD5OxnSLj0dWHqVPn-4yWR8C432JF8crhws9zmnprZOKwxpETTCwgWDz3zQfbjacJ_uX2bSEKTCASH71cp68DKAYGild4_Mi9NGnxMX7faBlph-Wj1B3qsvnjPGUxOamk8RcuPAG7lUl_imnYjnMApTl0JnBEZWut9XLBHJ6U8sHVm0aSv1yEYg3aUIk5QZcIz",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 42.0855146,
                    "lng": 48.2931943
                },
                "viewport": {
                    "northeast": {
                        "lat": 42.08687432989272,
                        "lng": 48.29463382989272
                    },
                    "southwest": {
                        "lat": 42.084174670

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 39.235014,
                    "lng": 42.860598
                },
                "viewport": {
                    "northeast": {
                        "lat": 39.23643377989272,
                        "lng": 42.86197867989272
                    },
                    "southwest": {
                        "lat": 39.23373412010728,
                        "lng": 42.85927902010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "GRAND S\u00dcPHAN OTEL PATNOS",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 500,
                    "html_attributions": [
                    

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp (F),Wind Speed,Hotel Name
0,kinango,0,KE,1627242345,82,-4.1372,39.3153,73.51,6.49,
1,yima,0,CN,1627242364,85,34.7381,111.8839,73.44,1.70,Huanghe Hotel
2,lithakia,0,GR,1627242389,50,37.7167,20.8333,79.88,1.14,Cavo Vezal
3,jishou,0,CN,1627242393,90,28.3167,109.7167,73.83,1.43,Xiangxi National Hotel
4,itambacuri,0,BR,1627242405,55,-18.0311,-41.6850,72.23,9.04,Pousada Bela
5,urumqi,0,CN,1627242135,26,43.8010,87.6005,79.75,6.71,Sheraton Urumqi Hotel
6,luanda,0,AO,1627242188,83,-8.8368,13.2343,71.60,6.91,EPIC SANA Luanda
7,derbent,0,RU,1627242441,67,42.0678,48.2899,77.54,7.00,Metropol'
8,patnos,0,TR,1627242442,26,39.2336,42.8634,70.23,0.18,GRAND SÜPHAN OTEL PATNOS
9,bezlyudivka,0,UA,1627242445,43,49.8718,36.2952,72.03,8.99,


In [33]:
hotel_df
hotel_df.dropna()
hotel_df.head(10)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp (F),Wind Speed,Hotel Name
0,kinango,0,KE,1627242345,82,-4.1372,39.3153,73.51,6.49,
1,yima,0,CN,1627242364,85,34.7381,111.8839,73.44,1.70,Huanghe Hotel
2,lithakia,0,GR,1627242389,50,37.7167,20.8333,79.88,1.14,Cavo Vezal
3,jishou,0,CN,1627242393,90,28.3167,109.7167,73.83,1.43,Xiangxi National Hotel
4,itambacuri,0,BR,1627242405,55,-18.0311,-41.6850,72.23,9.04,Pousada Bela
5,urumqi,0,CN,1627242135,26,43.8010,87.6005,79.75,6.71,Sheraton Urumqi Hotel
6,luanda,0,AO,1627242188,83,-8.8368,13.2343,71.60,6.91,EPIC SANA Luanda
7,derbent,0,RU,1627242441,67,42.0678,48.2899,77.54,7.00,Metropol'
8,patnos,0,TR,1627242442,26,39.2336,42.8634,70.23,0.18,GRAND SÜPHAN OTEL PATNOS
9,bezlyudivka,0,UA,1627242445,43,49.8718,36.2952,72.03,8.99,


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# Add marker layer ontop of heat map

info_box_content = hotel_info
markers = gmaps.marker_layer(locations, hotel_info)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display Map
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))